This notebook uses the OCaml 4.08.0 kernel

```
https://github.com/akabe/ocaml-jupyter
```

You can download this file and the required ocaml files to run it at http://ismailbennani.me/hybrid/diff_hybrid.zip

In [20]:
#require "jupyter.notebook" ;;
#directory ".";;

# Computing partial derivatives w.r.t. parameters in hybrid systems


Summary:
    - Automatic Differentiation of OCaml programs
    - Automatic Differentiation of Synchronous systems
    - Sensitivity analysis of Hybrid systems

## Automatic Differentiation of OCaml programs (FADBADml)
- forward differentiation: propagate partial derivatives in the DAG of the arithmetic expression from leaves to roots
- how ? carry an array of derivatives alongside values and override arithmetic operators to compute the correct array

In [2]:
type fad_float = {
    v : float;
    mutable diff : float array;
}

let ( +@ ) t1 t2 = {
    v = t1.v +. t2.v;
    diff = Array.map2 ( +. ) t1.diff t2.diff
}

let ( -@ ) t1 t2 = {
    v = t1.v -. t2.v;
    diff = Array.map2 ( -. ) t1.diff t2.diff
}

let ( *@ ) t1 t2 = {
    v = t1.v *. t2.v;
    diff = Array.map2 (fun d1 d2 -> d1 *. t2.v +. t1.v *. d2) t1.diff t2.diff;
}

let ( /@ ) t1 t2 = {
    v = t1.v /. t2.v;
    diff = Array.map2 (fun d1 d2 -> (d1 *. t2.v -. t1.v *. d2) /. (t2.v *. t2.v)) t1.diff t2.diff;
}

type fad_float = { v : float; mutable diff : float array; }


val ( +@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( -@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( *@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( /@ ) : fad_float -> fad_float -> fad_float = <fun>


- now that we know how to propagate the derivatives, how do we initialize them ?

In [3]:
let x = {
    v = 5.;
    diff = [| 1.; 0. |]
}

let y = {
    v = 2.;
    diff = [| 0.; 1. |]
}

let z = (x +@ y) *@ x

val x : fad_float = {v = 5.; diff = [|1.; 0.|]}


val y : fad_float = {v = 2.; diff = [|0.; 1.|]}


val z : fad_float = {v = 35.; diff = [|12.; 5.|]}


$$z(x,y) = (x + y) * x \qquad \frac{dz}{dx}(x,y) = 2x + y \qquad \frac{dz}{dy}(x,y) = x$$

- what about control structures ?

In [4]:
let z(x, y) =
    if x.v > 2. then
        x +@ y
    else 
        x *@ y

let z1 = z({ x with v = 0. }, { y with v = 5. })
let z2 = z({ x with v = 3. }, { y with v = 5. })

val z : fad_float * fad_float -> fad_float = <fun>


val z1 : fad_float = {v = 0.; diff = [|5.; 0.|]}


val z2 : fad_float = {v = 8.; diff = [|1.; 1.|]}


$$z(x,y) = \left\{\begin{array}{lr}x + y & \text{ if } x > 2 \\ x * y & \text{ else }\end{array}\right. \qquad , \qquad
\frac{dz}{dx}(x,y) = \left\{\begin{array}{lr}1 & \text{ if } x > 2 \\ y & \text{ else }\end{array}\right. \qquad , \qquad
\frac{dz}{dy}(x,y) = \left\{\begin{array}{lr}1 & \text{ if } x > 2 \\ x & \text{ else }\end{array}\right.
$$

---

For more conveniency, we allow the operands to have diff arrays of different sizes

In [5]:
let dim t = Array.length t.diff
let d t i = if i < dim t then t.diff.(i) else 0.

let ( ~-@ ) t = {
    v = -. t.v;
    diff = Array.map ( ~-. ) t.diff;
}

let ( +@ ) t1 t2 = {
    v = t1.v +. t2.v;
    diff = Array.init (max (dim t1) (dim t2)) 
        (fun i -> (d t1 i) +. (d t2 i));
}

let ( -@ ) t1 t2 = {
    v = t1.v -. t2.v;
    diff = Array.init (max (dim t1) (dim t2)) 
        (fun i -> (d t1 i) -. (d t2 i));
}

let ( *@ ) t1 t2 = {
    v = t1.v *. t2.v;
    diff = Array.init (max (dim t1) (dim t2)) 
        (fun i -> (d t1 i) *. t2.v +. t1.v *. (d t2 i));
}

let ( /@ ) t1 t2 = {
    v = t1.v /. t2.v;
    diff = Array.init (max (dim t1) (dim t2)) 
        (fun i -> ((d t1 i) *. t2.v -. t1.v *. (d t2 i)) /. (t2.v *. t2.v));
}

let cste f = { v = f; diff = [||]}
let var f i = { v = f; diff = Array.init (i+1) (fun k -> if k = i then 1. else 0.) }

val dim : fad_float -> int = <fun>


val d : fad_float -> int -> float = <fun>


val ( ~-@ ) : fad_float -> fad_float = <fun>


val ( +@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( -@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( *@ ) : fad_float -> fad_float -> fad_float = <fun>


val ( /@ ) : fad_float -> fad_float -> fad_float = <fun>


val cste : float -> fad_float = <fun>


val var : float -> int -> fad_float = <fun>


---

## Automatic Differentiation of Synchronous systems

Discrete node
* `alloc ()` allocate the initial state
* `step q i` compute the next state and current output using the current state and input `q` and `i`

In [6]:
type ('s, 'a, 'b) node = {
    alloc : unit -> 's;
    step : 's -> 'a -> 's * 'b;
}

type ('s, 'a, 'b) node = { alloc : unit -> 's; step : 's -> 'a -> 's * 'b; }


Example: free fall

In [7]:
type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool }

let g = cste 10.0
let h = cste 0.1

let free_fall = 
    let alloc () = {
        pre_y = cste 42.;
        pre_vy = cste 42.;
        initial = true;
    } in
    
    let step { pre_y; pre_vy; initial } y0 =
        if initial then
            { pre_y = y0; pre_vy = cste 0.; initial = false; }, y0
        else
            let vy = pre_vy -@ h *@ g in
            let y = pre_y +@ h *@ vy in
            { pre_y = y; pre_vy = vy; initial = false; }, y
    in
    
    { alloc; step }

type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool; }


val g : fad_float = {v = 10.; diff = [||]}


val h : fad_float = {v = 0.1; diff = [||]}


val free_fall : (mem, fad_float, fad_float) node =
  {alloc = <fun>; step = <fun>}


Simulate the free fall node and compute the partial derivatives of its output w.r.t. the initial state 

In [8]:
let _ =
    let y0 = var 5. 0 in
    let { alloc; step } = free_fall in
    let q = ref (alloc ()) in
    
    for i = 0 to 5 do
        let new_q, y = step !q y0 in
        
        Printf.printf "Step %d: y_%d = %g \t dy_%d/dy0 = %g\n" i i y.v i (d y 0);
        
        q := new_q;
    done;
    flush stdout;

Step 0: y_0 = 5 	 dy_0/dy0 = 1
Step 1: y_1 = 4.9 	 dy_1/dy0 = 1
Step 2: y_2 = 4.7 	 dy_2/dy0 = 1
Step 3: y_3 = 4.4 	 dy_3/dy0 = 1
Step 4: y_4 = 4 	 dy_4/dy0 = 1
Step 5: y_5 = 3.5 	 dy_5/dy0 = 1


- : unit = ()


Other example: free fall with drag, the drag coefficient `k` takes different values at each step

In [9]:
type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool }

let m = cste 1.

let free_fall_with_drag = 
    (* 
       y'' = - g - k/m * y'
       y(0) = y0 
       y'(0) = 0 
    *)

    let alloc () = {
        pre_y = cste 42.;
        pre_vy = cste 42.;
        initial = true;
    } in
    
    let step { pre_y; pre_vy; initial } (y0, k) =
        if initial then
            { pre_y = y0; pre_vy = cste 0.; initial = false }, y0
        else 
            let vy = pre_vy -@ h *@ (g +@ k /@ m *@ pre_vy) in
            let y = pre_y +@ h *@ vy in
            { pre_y = y; pre_vy = vy; initial = false }, y
    in
    
    { alloc; step }

type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool; }


val m : fad_float = {v = 1.; diff = [||]}


val free_fall_with_drag : (mem, fad_float * fad_float, fad_float) node =
  {alloc = <fun>; step = <fun>}


In [10]:
let k i = 1. +. (float i) /. 10.

let _ =
    let y0 = var 5. 0 in
    let { alloc; step } = free_fall_with_drag in
    let q = ref (alloc ()) in
    
    for i = 1 to 5 do
        let k = var (k i) 1 in
        let new_q, y = step !q (y0, k) in
        
        Printf.printf "Step %d: y_%d = %g \t dy_%d/dy0 = %g \t %s\n" i i y.v i (d y 0)
            (String.concat " \t " (Array.to_list (Array.mapi 
                (fun k d -> 
                    Printf.sprintf "dy_%d/dk_%d = %g" i k d;)
                (Array.sub y.diff 1 (dim y - 1)))));
        
        q := new_q;
    done;
    flush stdout

val k : int -> float = <fun>


Step 1: y_1 = 5 	 dy_1/dy0 = 1 	 
Step 2: y_2 = 4.9 	 dy_2/dy0 = 1 	 dy_2/dk_0 = 0
Step 3: y_3 = 4.713 	 dy_3/dy0 = 1 	 dy_3/dk_0 = 0.01
Step 4: y_4 = 4.45218 	 dy_4/dy0 = 1 	 dy_4/dk_0 = 0.0373
Step 5: y_5 = 4.13048 	 dy_5/dy0 = 1 	 dy_5/dk_0 = 0.086587


- : unit = ()


## Sensitivity w.r.t. parameters in Hybrid systems

A hybrid program is a discrete program that may execute an ODE solver between two discrete steps

### Sundials (CVodes)

$$\left\{\begin{array}{l}y' = f(t, y, p)\\y(0) = y_0(p)\end{array}\right.$$

`p` is a parameter vector, its values do not depend on `t`

Cvode approximates the solution y.

CVodes approximates the sensitivities of y w.r.t. some predefined parameters:
$$s_i = \frac{\partial y}{\partial p_i}$$

Forward sensitivity analysis: by deriving the system above w.r.t. some $p_i$, we get:
$$\dot{s_i} = \frac{\partial f}{\partial y} s_i + \frac{\partial f}{\partial p_i} \qquad , \qquad s_i(0) = \frac{\partial y_0}{\partial p_i}$$

### Solver interface

In [11]:
module type Solver =
sig
    type session
    type time = float
    type params = float array
    type nvec = float array       (* state vector *)
    type smat = float array array (* sensitivity matrix *)
    
    (* [f : rhsfn] is such that [y' = f params t y] *)
    type rhsfn = params -> time -> nvec -> nvec
    
    (* [init f params nvec smat] creates a solver session with function [f], 
       parameters [params], initial state [nvec] and initial sensitivity matrix [smat] *)
    val init : rhsfn -> params -> nvec -> smat -> session
    val reinit : session -> time -> nvec -> session
    
    (* [t' = step s t] performs one integration step with max horizon [t]. 
        It returns the time [t'] at which the integration stoped *)
    val step : session -> time -> time
    
    (* [get_dky s t i] returns [c] the i-th derivative of the state vector at time [t] *)
    val get_dky : session -> time -> int -> nvec
    
    (* [get_sdky s t i] returns [smat] the i-th derivatives of the state vector 
       with respect to the parameters at time [t] *)
    val get_sdky : session -> time -> int -> smat
end

module type Solver =
  sig
    type session
    type time = float
    type params = float array
    type nvec = float array
    type smat = float array array
    type rhsfn = params -> time -> nvec -> nvec
    val init : rhsfn -> params -> nvec -> smat -> session
    val reinit : session -> time -> nvec -> session
    val step : session -> time -> time
    val get_dky : session -> time -> int -> nvec
    val get_sdky : session -> time -> int -> smat
  end


### Hybrid system

- a hybrid system is a discrete system that produces an ODE system at each step
- for the sake of simplicity, our inputs and outputs will be of type `float array`

To run the following pieces of code, please install `sundialsml`:
```
opam install sundialsml
```

then download `sundials_cvodes.ml` from [zelus repository](https://github.com/INRIA/zelus) in `lib/solver`, and put it in the same directory as this jupyter file. 

In [21]:
#use "topfind";;
#require "sundialsml";;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


In [13]:
#mod_use "ztypes.mli";;
#mod_use "zls.ml";;
#mod_use "sundials_cvodes.ml";;

module Ztypes :
  sig
    type 'a continuous = { mutable pos : 'a; mutable der : 'a; }
    type ('a, 'b) zerocrossing = { mutable zin : 'a; mutable zout : 'b; }
    type 'a signal = 'a * bool
    type zero = bool
    type ('a, 'b) node =
        Node : { alloc : unit -> 's; step : 's -> 'a -> 'b;
          reset : 's -> unit;
        } -> ('a, 'b) node
    type ('a, 'b) cnode =
        Cnode : { alloc : unit -> 's; copy : 's -> 's -> unit;
          step : 's -> 'a -> 'b; reset : 's -> unit;
        } -> ('a, 'b) cnode
    type time = float
    type cvec =
        (float, Bigarray.float64_elt, Bigarray.c_layout) Bigarray.Array1.t
    type dvec =
        (float, Bigarray.float64_elt, Bigarray.c_layout) Bigarray.Array1.t
    type zinvec =
        (int32, Bigarray.int32_elt, Bigarray.c_layout) Bigarray.Array1.t
    type zoutvec =
        (float, Bigarray.float64_elt, Bigarray.c_layout) Bigarray.Array1.t
    type cstate = {
      mutable dvec : dvec;
      mutable cvec : cvec;
      mutabl

In [14]:
#mod_use "solver.ml";;

module Solver :
  sig
    module RealArray = Sundials.RealArray
    type time = float
    type params = float array
    type nvec = float array
    type smat = float array array
    type rhsfn = params -> time -> nvec -> nvec
    type t = {
      nvec : Sundials_cvodes.nvec;
      smat : Sundials_cvodes.sensmat;
      session : Sundials_cvodes.t;
    }
    val init :
      (float array -> float -> float array -> float array) ->
      float array -> float array -> float array array -> t
    val reinit : t -> float -> float array -> float array array -> t
    val step : t -> float -> float
    val get_dky : t -> float -> int -> float array
    val arrays_of_sensmat : Nvector_serial.t array -> float array array
    val get_sdky : t -> float -> int -> float array array
  end


In [15]:
(* no fad_float in [derivatives]: the solvers only deals with float values *)
type ode = {
    initial_values : fad_float array;
    horizon : float; 
    derivatives : Solver.params -> float -> Solver.nvec -> Solver.nvec;
}

type 's hnode = {
    alloc : unit -> 's;
    
    (* [ode, reinit = ode s] computes an ode system and a [reinit] flag that
       instructs the ode solver to reinitialize itself. It will be true 
       if there is a discontinuity in the dynamic *)
    ode : 's -> ode * bool;
    
    (* [q', o = step q c z u] with [q] the current state, [c] the last state vector 
       computed by the solver (initially empty), [u] the current input, [q'] a new state
       and [o] the current output *)
    step : 's -> fad_float array -> fad_float array -> 's * fad_float array;
}

type ode = {
  initial_values : fad_float array;
  horizon : float;
  derivatives : Solver.params -> float -> Solver.nvec -> Solver.nvec;
}


type 's hnode = {
  alloc : unit -> 's;
  ode : 's -> ode * bool;
  step : 's -> fad_float array -> fad_float array -> 's * fad_float array;
}


In [16]:
type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool }

let g = 10.

let free_fall_c =
    let alloc () = {
        pre_y = cste 42.;
        pre_vy = cste 42.;
        initial = true;
    } in
    
    let ode { pre_y; pre_vy; initial } = {
        initial_values = [| pre_y; pre_vy |];
        horizon = infinity;
        derivatives = fun _ _ c -> [| c.(1); -. g |];
    }, initial in
    
    (* [u] only contains [y0] the initial height of the falling item,
       [c] will contain [y, vy] the height and speed computed by the solver *)
    let step { pre_y; initial } c u =
        if initial then
            (* instant initial *)
            { pre_y = u.(0); pre_vy = cste 0.; initial = false }, [| u.(0) |]
        else
            { pre_y = c.(0); pre_vy = c.(1); initial = false }, [| c.(0) |]
    in
    { alloc; ode; step }        

type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool; }


val g : float = 10.


val free_fall_c : mem hnode = {alloc = <fun>; ode = <fun>; step = <fun>}


### How to simulate such a program ?

We have:
- parameters $(p_i)$ defined using `var`

We need: 
$$s_i(0) = \frac{\partial y_0}{\partial p_i}$$

Here, the initial values of the state vectors are computed by `ode`, their partial derivatives will be computed alongside their values and accessible in the `initial_values` field of our ode record.

In [17]:
let h = 0.1

let _ = 
    let y0 = var 5. 0 in
    let nparams = 1 in
    let { alloc; step; ode } = free_fall_c in
    
    (* initial step *)
    let u = [| y0 |] in
    let new_q, out = step (alloc ()) [||] u in
    
    (* get initial ode system *)
    let { initial_values; horizon; derivatives }, _ = ode new_q in
    
    (* build initial state vector and initial sensitivity mat *)
    let nvec0 = (Array.map (fun f -> f.v) initial_values) in
    let smat0 = Array.init nparams
        (fun i -> 
            Array.init (Array.length initial_values)
                (fun j -> 
                    d initial_values.(j) i
                )
        )
    in

    (* initialize solver *)
    (* parameters are the inputs of the hybrid node *)
    let s = ref (Solver.init derivatives (Array.map (fun t -> t.v) u) nvec0 smat0) in
    
    let q = ref new_q in
    let i = ref 0 in
    let t = ref 0. in
    
    while !i < 10 do
        let cur_h = min horizon h in
        
        let new_t = Solver.step !s cur_h in
        
        let nvec = Solver.get_dky !s new_t 0 in
        let smat = Solver.get_sdky !s new_t 0 in
        
        (* the derivatives of state vector cell i is the i-th column of diff *)
        let c = Array.mapi 
            (fun i v ->
                let diff = Array.map (fun diff -> diff.(i)) smat in
                { v; diff }
            )
            nvec 
        in
        
        let new_q, out = step !q c u in
        
        Printf.printf "Step %d: t = %7.2g \t y = %11.2f \t dy/dy0 = %g\n" !i !t out.(0).v (d out.(0) 0);
        flush stdout;
        
        let { initial_values; horizon; derivatives }, reinit = ode new_q in
        
        if reinit then begin
            let nvec0 = (Array.map (fun f -> f.v) initial_values) in
            let smat0 = (Array.map (fun f -> f.diff) initial_values) in
            s := Solver.reinit !s !t nvec0 smat0
        end;
            
        q := new_q;
        t := !t +. new_t;
        i := !i + 1;
    done

val h : float = 0.1


Step 0: t =       0 	 y =        5.00 	 dy/dy0 = 1
Step 1: t = 1.9e-11 	 y =        5.00 	 dy/dy0 = 1
Step 2: t = 1.9e-07 	 y =        5.00 	 dy/dy0 = 1
Step 3: t = 2.3e-06 	 y =        5.00 	 dy/dy0 = 1
Step 4: t = 2.4e-05 	 y =        5.00 	 dy/dy0 = 1
Step 5: t = 0.00024 	 y =        5.00 	 dy/dy0 = 1
Step 6: t =  0.0024 	 y =        5.00 	 dy/dy0 = 1
Step 7: t =   0.024 	 y =        4.77 	 dy/dy0 = 1
Step 8: t =    0.24 	 y =      -18.00 	 dy/dy0 = 1
Step 9: t =     2.4 	 y =    -2294.56 	 dy/dy0 = 1


- : unit = ()


The paremters given to Sundials_cvodes.init are always the inputs of our nodes => this way, we do not need to know the actual parameters that the user wants to use at compile time, but we will need to apply the chain rule to convert from derivatives w.r.t. our parameters to derivatives w.r.t the inputs of the node.

In the example above, the parameter `y0` that we use are also the inputs of our node.

Let's take another example to show the more general case.

In [18]:
type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool }

let g = 10.
let pi = 3.14

let free_fall_c =
    let alloc () = {
        pre_y = cste 0.;
        pre_vy = cste 0.;
        initial = true;
    } in
    
    let ode { pre_y; pre_vy; initial } =
        {
            initial_values = [| pre_y; pre_vy |];
            horizon = infinity;
            derivatives = 
                fun params t c -> 
                    let k = params.(1) in
                    let m = params.(2) in
                    let vy = c.(1) in
                    
                    let ay = -. g -. k /. m *. vy in
                    
                    [| vy; ay |];
        }, 
        initial 
    in
    
    (* u will contain [y0, k, m],
       c will contain y and vy *)
    let step { pre_vy; initial } c u =
        let y0 = u.(0) in
        
        let y = if initial then y0 else c.(0) in
        let vy = if initial then cste 0. else c.(1) in
        
        { pre_y = y; pre_vy = vy; initial = false }, [| y |]
    in
    { alloc; ode; step }    

type mem = { pre_y : fad_float; pre_vy : fad_float; initial : bool; }


val g : float = 10.


val pi : float = 3.14


val free_fall_c : mem hnode = {alloc = <fun>; ode = <fun>; step = <fun>}


   
In the general case, we have an ODE system that looks like:
$$\left\{\begin{array}{l}y' = f(t, y, u(p)) \\ y'(0) = y_0(u(p))\end{array}\right.$$
    
The sensitivity analysis of Sundials' Cvodes will be done w.r.t. the values $u(p)$ of the inputs.

Reminder: we need to provide the initial sensitivities, here $$s_i(0) = \frac{\partial y_0}{\partial u_i}$$ where $u_i$ is the $i$-th coordinate of $u(p)$.

To compute that, we will:
- copy the array `u` into a new array and change the `diff` field of its cells such that the partial derivatives are computed w.r.t. the values of the inputs instead of those of the parameters. In practice it consists of replacing the `diff` array of the i-th element by $[0 \ldots 1 \ldots 0]$ the array that has $1$ in the i-th cell and $0$ elsewhere.
- call `step` and `òde` with this new array of inputs
- get the sensitivities computed by sundials, that is $\frac{\partial y}{\partial u_i}$
- use the derivatives $\frac{\partial u_i}{\partial p_k}$ to compute the partial derivatives that we are interested in: $\frac{\partial y}{\partial p_k}$
$$\frac{\partial y}{\partial p_k} = \sum_i \frac{\partial y}{\partial u_i} \frac{\partial u_i}{\partial p_k}$$

In [19]:
let h = 0.1

let k eta = cste (6. *. pi) *@ eta

let _ = 
    let y0 = var 5. 0 in
    let eta = var 2. 1 in
    let m = var 10. 2 in    
    
    (* on calcule l'entrée k *)
    
    let k = k eta in
    
    let { alloc; step; ode } = free_fall_c in
    
    (* initial step *)
    
    (* entrée u *)
    let u = [| y0; k; m |] in
    
    (* compute the number of parameters *)
    let nparams = 3 in
    
    let v = Array.mapi (fun i t -> {
        v = t.v;
        diff = Array.init (Array.length u) (fun j -> if i = j then 1. else 0.)
    }) u in
    
    (* here, out will contain the partial derivatives w.r.t. the values in v
       because the [diff] arrays of the elements of v look like [0,...,1,...,0] *)
    let new_q, out = step (alloc ()) [||] v in
    
    (* get initial ode system *)
    let { initial_values; horizon; derivatives }, _ = ode new_q in
    
    print_endline "Initial values and partial derivatives";
    Array.iteri (fun i t -> Printf.printf "%d: %f, [%s]\n" i t.v (String.concat ", " (Array.to_list (Array.map string_of_float t.diff)))) initial_values;
    print_endline "";
    
    (* build initial state vector and initial sensitivity mat *)
    let nvec0 = (Array.map (fun f -> f.v) initial_values) in
    let smat0 = Array.init nparams
        (fun i -> 
            Array.init (Array.length initial_values)
                (fun j -> 
                    d initial_values.(j) i
                )
        )
    in
    
    print_endline "Initial nvec and smat";
    Printf.printf "Nvec:\n\t[%s]\n" (String.concat ", " (Array.to_list (Array.map string_of_float nvec0)));
    print_endline "Smat:";
    Array.iteri (fun i t -> Printf.printf "\t[%s]\n" (String.concat ", " (Array.to_list (Array.map string_of_float t)))) smat0;
    print_endline "";

    (* initialize solver *)
    let s = ref (Solver.init derivatives (Array.map (fun t -> t.v) v) nvec0 smat0) in
    
    let q = ref new_q in
    let i = ref 0 in
    let t = ref 0. in
    
    while !i < 10 do
        Printf.printf "---- Step %d, start time: %g\n" !i !t;
        
        let cur_h = min horizon h in
        
        let new_t = Solver.step !s cur_h in
        
        t := !t +. new_t;
        Printf.printf "New time: %g\n" !t;
        
        let nvec = Solver.get_dky !s new_t 0 in
        let smat = Solver.get_sdky !s new_t 0 in
        
        Printf.printf "Nvec:\n\t[%s]\n" (String.concat ", " (Array.to_list (Array.map string_of_float nvec)));
        print_endline "Smat:";
        Array.iteri (fun i t -> Printf.printf "\t[%s]\n" (String.concat ", " (Array.to_list (Array.map string_of_float t)))) smat;
        print_endline "";
        
        let c = Array.mapi (fun i v -> 
            let diff = Array.init nparams
                (fun j -> smat.(j).(i))
            in { v; diff }
        ) nvec 
        in
        
        print_endline "Carray given to step";
        Array.iteri (fun i t -> Printf.printf "\t%d: %f, [%s]\n" i t.v (String.concat ", " (Array.to_list (Array.map string_of_float t.diff)))) c;
        print_endline "";
        
        (* here, out contains the derivatives w.r.t. the elements of v
           because the diff fields of the elements of v look like [0,...,1,...,0] *)
        let new_q, out = step !q c v in
        
        print_endline "Output of step (with derivatives w.r.t. the inputs of step)";
        Array.iteri (fun i t -> Printf.printf "\t%d: %f, [%s]\n" i t.v (String.concat ", " (Array.to_list (Array.map string_of_float t.diff)))) out;
        print_endline "";
        
        let real_out = Array.mapi
            (fun i t -> 
                let v = t.v in
                let diff = Array.init nparams
                    (fun j -> 
                        let s = ref 0. in
                        (* the dy/du are in t.diff, the du/dp are in the diff field of the elements of u *)
                        for k = 0 to Array.length u - 1 do
                            s := !s +. (d u.(k) j) *. (d t k)
                        done;
                        !s
                    )
                in { v; diff }
            ) out
        in
        
        print_endline "Output of step (with derivatives w.r.t. the parameters y0, eta and m)";
        Array.iteri (fun i t -> Printf.printf "\t%d: %f, [%s]\n" i t.v (String.concat ", " (Array.to_list (Array.map string_of_float t.diff)))) real_out;
        print_endline "";
        
        Printf.printf "Output: t=%g \t y = %g \t dy/dy0 = %g \t dy/deta = %g \t dy/dm = %g\n\n" 
            !t 
            real_out.(0).v 
            (d real_out.(0) 0) (d real_out.(0) 1) (d real_out.(0) 2);
        flush stdout;
        
        let { initial_values; horizon; derivatives }, reinit = ode new_q in
        
        if reinit then begin
            let nvec0 = (Array.map (fun f -> f.v) initial_values) in
            let smat0 = (Array.map (fun f -> f.diff) initial_values) in
            s := Solver.reinit !s !t nvec0 smat0
        end;
            
        q := new_q;
        i := !i + 1;
    done

val h : float = 0.1


val k : fad_float -> fad_float = <fun>


Initial values and partial derivatives
0: 5.000000, [1., 0., 0.]
1: 0.000000, []

Initial nvec and smat
Nvec:
	[5., 0.]
Smat:
	[1., 0.]
	[0., 0.]
	[0., 0.]

---- Step 0, start time: 0
New time: 1.9301e-11
Nvec:
	[5., -1.9301011108e-10]
Smat:
	[1., 0.]
	[0., 3.71997491544e-22]
	[0., -1.4039905326e-21]

Carray given to step
	0: 5.000000, [1., 0., 0.]
	1: -0.000000, [0., 3.71997491544e-22, -1.4039905326e-21]

Output of step (with derivatives w.r.t. the inputs of step)
	0: 5.000000, [1., 0., 0.]

Output of step (with derivatives w.r.t. the parameters y0, eta and m)
	0: 5.000000, [1., 0., 0.]

Output: t=1.9301e-11 	 y = 5 	 dy/dy0 = 1 	 dy/deta = 0 	 dy/dm = 0

---- Step 1, start time: 1.9301e-11
New time: 1.93049e-07
Nvec:
	[5., -1.93029271722e-06]
Smat:
	[1., 0.]
	[7.193380623e-25, 3.72566291428e-14]
	[-2.71386632595e-24, -1.40383111914e-13]

Carray given to step
	0: 5.000000, [1., 7.193380623e-25, -2.71386632595e-24]
	1: -0.000002, [0., 3.72566291428e-14, -1.40383111914e-13]

Output of s

- : unit = ()
